In [1]:
import pandas as pd
import plotly.express as px

df_index_data = pd.read_csv("datasets/indexData.csv")
# print(df_index_data.head())

df_index_info = pd.read_csv('datasets/indexInfo.csv')
# print(df_index_info.head())

df_index_processed = pd.read_csv('datasets/indexProcessed.csv')
# print(df_index_processed.head())

print(df_index_data.describe())
print(df_index_info.describe())
print(df_index_processed.describe())


                Open           High            Low          Close  \
count  110253.000000  110253.000000  110253.000000  110253.000000   
mean     7658.515222    7704.372961    7608.000422    7657.545872   
std      9011.478913    9066.638548    8954.506981    9011.510444   
min        54.869999      54.869999      54.869999      54.869999   
25%      1855.030029    1864.510010    1843.979980    1855.060059   
50%      5194.089844    5226.279785    5154.049805    5194.750000   
75%     10134.299810   10207.820310   10060.349610   10134.830080   
max     68775.062500   69403.750000   68516.992190   68775.062500   

           Adj Close        Volume  
count  110253.000000  1.102530e+05  
mean     7657.351729  1.273975e+09  
std      9011.608900  4.315783e+09  
min        54.869999  0.000000e+00  
25%      1854.179565  0.000000e+00  
50%      5194.750000  4.329000e+05  
75%     10134.830080  1.734314e+08  
max     68775.062500  9.440374e+10  
               Region                 Exchang

In [2]:
# find rows in df_index_data but not in df_index_processed
diff = pd.concat([df_index_data, df_index_processed[:-2]]).drop_duplicates(keep=False)
print(diff)

          Index        Date          Open          High           Low  \
0           NYA  1965-12-31    528.690002    528.690002    528.690002   
1           NYA  1966-01-03    527.210022    527.210022    527.210022   
2           NYA  1966-01-04    527.840027    527.840027    527.840027   
3           NYA  1966-01-05    531.119995    531.119995    531.119995   
4           NYA  1966-01-06    532.070007    532.070007    532.070007   
...         ...         ...           ...           ...           ...   
104217  J203.JO  2021-05-21  66124.437500  66771.343750  66124.437500   
104218  J203.JO  2021-05-24  66238.929690  66493.132810  65940.257810   
104219  J203.JO  2021-05-25  66054.921880  66812.453130  66022.976560   
104220  J203.JO  2021-05-26  66076.679690  66446.367190  66030.351560   
104221  J203.JO  2021-05-27  66108.226560  66940.250000  66102.546880   

               Close     Adj Close  Volume     CloseUSD  
0         528.690002    528.690002     0.0          NaN  
1      

In [3]:
df_index_info_without_currency = df_index_info.drop(columns='Currency')

df_all_info = pd.merge(df_index_processed, df_index_info_without_currency, how='left', on='Index')
print(df_all_info)

          Index        Date          Open          High           Low  \
0           HSI  1986-12-31   2568.300049   2568.300049   2568.300049   
1           HSI  1987-01-02   2540.100098   2540.100098   2540.100098   
2           HSI  1987-01-05   2552.399902   2552.399902   2552.399902   
3           HSI  1987-01-06   2583.899902   2583.899902   2583.899902   
4           HSI  1987-01-07   2607.100098   2607.100098   2607.100098   
...         ...         ...           ...           ...           ...   
104219  J203.JO  2021-05-25  66054.921880  66812.453130  66022.976560   
104220  J203.JO  2021-05-26  66076.679690  66446.367190  66030.351560   
104221  J203.JO  2021-05-27  66108.226560  66940.250000  66102.546880   
104222  J203.JO  2021-05-28  66940.250000  67726.562500  66794.609380   
104223  J203.JO  2021-05-31  67554.859380  68140.851560  67554.859380   

               Close     Adj Close  Volume     CloseUSD        Region  \
0        2568.300049   2568.300049     0.0   333.8

In [4]:
# Data Cleaning and Preprocessing

null_count = df_index_processed.isnull().sum()
print("Null count \n", null_count)

rows_with_nulls = df_index_processed[df_index_processed.isnull().any(axis=1)]
print("rows with nulls \n", rows_with_nulls)

df_index_processed["Date"] = pd.to_datetime(df_index_processed["Date"], errors="coerce")
invalid_dates = df_index_processed[df_index_processed["Date"].isnull()]
print("Invalid dates:\n", invalid_dates)

Null count 
 Index        0
Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
CloseUSD     0
dtype: int64
rows with nulls 
 Empty DataFrame
Columns: [Index, Date, Open, High, Low, Close, Adj Close, Volume, CloseUSD]
Index: []
Invalid dates:
 Empty DataFrame
Columns: [Index, Date, Open, High, Low, Close, Adj Close, Volume, CloseUSD]
Index: []


In [5]:

# Basic Data Exploration

# What are the mean, median, and standard deviation of the closing prices for each stock exchange?
closeUSD_agg_func = {'CloseUSD': ['mean', 'median', 'std']}
df_exchange_closeUSD__stats = df_all_info.groupby(by=['Exchange']).agg(closeUSD_agg_func).reset_index()
df_exchange_closeUSD__stats.columns = ['Exchange', 'MeanCloseUSD', 'MedianCloseUSD', 'STDCloseUSD']
print(df_exchange_closeUSD__stats)

# What are the minimum and maximum values for the opening prices across all exchanges?
open_agg_func = {'Open': ['min', 'max']}
df_change_open_stats = df_all_info.groupby(by=['Exchange']).agg(open_agg_func).reset_index()
df_change_open_stats.columns = ['Exchange', 'MinOpen', 'MaxOpen']
print(df_change_open_stats)




                            Exchange  MeanCloseUSD  MedianCloseUSD  \
0                           Euronext   1003.027537     1011.166515   
1           Frankfurt Stock Exchange   7216.112995     6589.354033   
2           Hong Kong Stock Exchange   1976.078732     1846.016885   
3        Johannesburg Stock Exchange   3563.766210     3652.466016   
4                             NASDAQ   1984.906795     1091.444946   
5   National Stock Exchange of India     76.600472       74.853250   
6            New York Stock Exchange   4452.174711     2631.800049   
7                 SIX Swiss Exchange   7114.803716     7380.501109   
8            Shanghai Stock Exchange    381.291062      362.892812   
9            Shenzhen Stock Exchange   1275.812961     1342.891250   
10             Taiwan Stock Exchange    321.154646      316.831992   
11              Tokyo Stock Exchange    128.491552      117.859150   
12            Toronto Stock Exchange   6714.755033     5898.436540   

    STDCloseUSD  
0

In [6]:
df_index_skew = df_all_info.groupby(by='Exchange').agg({'CloseUSD': 'skew'}).reset_index()
df_index_skew.columns = ['Exchange', 'SkewCloseUSD']
df_index_skew_top5 = df_index_skew.sort_values(by='SkewCloseUSD', ascending=False, ignore_index=True).head(5)
print(df_index_skew_top5)

df_index_kurt = df_all_info[['Exchange', 'CloseUSD']].groupby(by='Exchange').apply(pd.DataFrame.kurt).reset_index()
df_index_kurt.columns = ['Exchange', 'KurtCloseUSD']
df_index_kurt_top5 = df_index_kurt.sort_values(by='KurtCloseUSD', ascending=False, ignore_index=True).head(5)
print(df_index_kurt_top5)

# df_index_skew_kurt = df_index_skew.merge(df_index_kurt, left_on='Exchange', right_on='Exchange')
# print(df_index_skew_kurt)

                                            




                   Exchange  SkewCloseUSD
0                    NASDAQ      2.110524
1     Taiwan Stock Exchange      0.878606
2   New York Stock Exchange      0.736671
3   Shanghai Stock Exchange      0.721964
4  Frankfurt Stock Exchange      0.582135
                      Exchange  KurtCloseUSD
0                       NASDAQ      5.034252
1        Taiwan Stock Exchange      1.922342
2      Shanghai Stock Exchange      0.615299
3  Johannesburg Stock Exchange      0.108165
4         Tokyo Stock Exchange     -0.297929


In [7]:
# Time Series Analysis
# Compute the average daily closing prices for the US stock exchange over the last 5 years

df_all_info['Date'] = pd.to_datetime(df_all_info['Date'])
five_years_ago = df_all_info['Date'].max() - pd.DateOffset(years=5)

df_us_index_last_5y = df_all_info[(df_all_info['Region'] =='United States') & (df_all_info["Date"] >= five_years_ago)]

avg_us_daily_close_last_5y = df_us_index_last_5y['Close'].mean()
print(avg_us_daily_close_last_5y)

10310.336845823647


In [11]:
# Identify any significant trends or patterns in the time series data for the Japanese stock exchange by analyzing the statistical summary.c

df_jp_index = df_all_info[df_all_info['Region'] =='Japan']
print(df_jp_index.describe())

fig = px.line(df_jp_index, x='Date', y='Close', title='Japan Index Overtime')
fig.show()

                                Date          Open          High  \
count                          13874  13874.000000  13874.000000   
mean   1993-03-06 11:56:28.265820928  12852.286239  12919.694377   
min              1965-01-05 00:00:00   1020.489990   1020.489990   
25%              1979-02-22 06:00:00   6080.977417   6080.977417   
50%              1993-02-25 12:00:00  11802.810055  11848.575195   
75%              2007-03-25 06:00:00  18559.502932  18675.247070   
max              2021-06-03 00:00:00  38921.648440  38957.441410   
std                              NaN   8028.559838   8077.374703   

                Low         Close     Adj Close        Volume      CloseUSD  
count  13874.000000  13874.000000  13874.000000  1.387400e+04  13874.000000  
mean   12778.636383  12849.155197  12849.155197  3.976289e+07    128.491552  
min     1020.489990   1020.489990   1020.489990  0.000000e+00     10.204900  
25%     6080.977417   6080.977417   6080.977417  0.000000e+00     60.809774

In [25]:
# Compare the average daily trading volume between the US and Chinese stock exchanges
df_avg_daily_trading_us_cn = df_all_info[df_all_info['Region'].isin(['United States', 'China'])]
avg_daily_trading_vol = df_avg_daily_trading_us_cn[['Region', 'Volume']].groupby(by=['Region']).mean()


print(avg_daily_trading_vol)

                     Volume
Region                     
China          1.780912e+05
United States  1.108833e+09
